In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import numpy.lib.stride_tricks as st

# --------------------------------------------------
# Paths and basic configuration
# --------------------------------------------------
PATH = "/content/drive/MyDrive/Datamining-TSC-Project/new_processed_data.parquet"
CFG = {
    "time_col": "time",
    "window": 36,        # sliding window length (hours)
    "trend_h": 12,       # recent hours for trend checks
    "ma_hours": [3, 6, 12],  # moving average windows
}

# --------------------------------------------------
# Angle utility functions
# --------------------------------------------------
def wrap360(x):
    # Wrap angles into [0, 360)
    return (x % 360.0 + 360.0) % 360.0

def angle_diff_deg(a, b):
    # Smallest signed angle difference a - b in degrees
    return (a - b + 180.0) % 360.0 - 180.0

def wave_dir_convert(old_wave_dir):
    # Convert wave direction to wind-direction convention
    return wrap360(270.0 - old_wave_dir)

# --------------------------------------------------
# Moving average feature generator
# --------------------------------------------------
def add_moving_averages(df, cols, ma_hours):
    # Add rolling mean features for selected columns
    for col in cols:
        for h in ma_hours:
            df[f"{col}_ma{h}"] = df[col].rolling(window=h, min_periods=h).mean()
    return df

# --------------------------------------------------
# Load and clean data
# --------------------------------------------------
df = pd.read_parquet(PATH)

df["time"] = pd.to_datetime(df["time"])
df = df.sort_values("time").drop_duplicates("time").reset_index(drop=True)

# Keep only relevant columns
df = df[
    [
        "time",
        "Wind speed",
        "Wind Direction",
        "Wave Period",
        "Wave Direction",
        "Wave Height",
        "Wave Power",
        "Pressure",
        "temperature",
        "Surge Height",
        "Total Water Level",
        "Wave Steepness",
    ]
].copy()

# Rename columns to short, consistent names
df.rename(
    columns={
        "Wind speed": "ws",
        "Wind Direction": "wd",
        "Wave Period": "tp",
        "Wave Direction": "wdir",
        "Wave Height": "hs",
        "Wave Power": "pwr",
        "Pressure": "mslp",
        "temperature": "temp",
        "Surge Height": "surge",
        "Total Water Level": "twl",
        "Wave Steepness": "steep",
    },
    inplace=True,
)

# --------------------------------------------------
# Wind–wave direction alignment features
# --------------------------------------------------
df["wdir"] = wave_dir_convert(df["wdir"].to_numpy(np.float32))

wd = df["wd"].to_numpy(np.float32)
wdir = df["wdir"].to_numpy(np.float32)

# Angle difference between wind and wave directions
dwd_deg = angle_diff_deg(wd, wdir).astype(np.float32)
dwd_rad = np.deg2rad(dwd_deg).astype(np.float32)

# Encode direction difference with sin/cos
df["dwd_sin"] = np.sin(dwd_rad).astype(np.float32)
df["dwd_cos"] = np.cos(dwd_rad).astype(np.float32)

# Drop raw direction columns
df.drop(columns=["wd", "wdir"], inplace=True)

# --------------------------------------------------
# Add moving average features
# --------------------------------------------------
ma_cols = [
    "hs", "ws", "pwr", "mslp",
    "temp", "surge", "twl", "steep", "tp"
]
df = add_moving_averages(df, ma_cols, CFG["ma_hours"])

# --------------------------------------------------
# Sliding window statistics
# --------------------------------------------------
W = CFG["window"]
H = CFG["trend_h"]

hs   = df["hs"].to_numpy(np.float32)
pwr  = df["pwr"].to_numpy(np.float32)
mslp = df["mslp"].to_numpy(np.float32)
ws   = df["ws"].to_numpy(np.float32)
dwd_cos = df["dwd_cos"].to_numpy(np.float32)

# Create rolling windows
hs_w   = st.sliding_window_view(hs,   W)
pwr_w  = st.sliding_window_view(pwr,  W)
mslp_w = st.sliding_window_view(mslp, W)
ws_w   = st.sliding_window_view(ws,   W)
dwd_cos_w = st.sliding_window_view(dwd_cos, W)

# Window-based severity metrics (mean + 2*std)
hs_metric36  = hs_w.mean(axis=1)  + 2.0 * hs_w.std(axis=1)
pwr_metric36 = pwr_w.mean(axis=1) + 2.0 * pwr_w.std(axis=1)

# --------------------------------------------------
# Train / validation / test split by time
# --------------------------------------------------
start_times = df["time"].iloc[:len(hs_metric36)].to_numpy()

train_mask = start_times < np.datetime64("2015-01-01")
val_mask   = (start_times >= np.datetime64("2015-01-01")) & (start_times < np.datetime64("2020-01-01"))
test_mask  = start_times >= np.datetime64("2020-01-01")

# --------------------------------------------------
# Percentile-based severity thresholds (train only)
# --------------------------------------------------
hs_p75, hs_p92, hs_p98, hs_p995 = np.percentile(
    hs_metric36[train_mask], [75, 92, 98, 99.5]
)
pwr_p75, pwr_p92, pwr_p98, pwr_p995 = np.percentile(
    pwr_metric36[train_mask], [75, 92, 98, 99.5]
)

# Map continuous values to severity classes
def severity(x, p75, p92, p98, p995):
    y = np.zeros_like(x, dtype=np.int8)
    y[(x >= p75) & (x < p92)]  = 1
    y[(x >= p92) & (x < p98)]  = 2
    y[(x >= p98) & (x < p995)] = 3
    y[(x >= p995)]             = 4
    return y

sev_hs  = severity(hs_metric36,  hs_p75,  hs_p92,  hs_p98,  hs_p995)
sev_pwr = severity(pwr_metric36, pwr_p75, pwr_p92, pwr_p98, pwr_p995)

# Base severity: worst of wave height or power
base = np.maximum(sev_hs, sev_pwr)

# --------------------------------------------------
# Trend-based reinforcement rules (train only)
# --------------------------------------------------
train_hours_mask = df["time"] < "2015-01-01"

hs_th   = np.percentile(hs[train_hours_mask],  92)
pwr_th  = np.percentile(pwr[train_hours_mask], 92)
ws_th   = np.percentile(ws[train_hours_mask],  92)
mslp_th = np.percentile(mslp[train_hours_mask], 20)
align_th = np.percentile(dwd_cos[train_hours_mask], 75)

# Count how many storm-like conditions persist in last H hours
cnt = (
    (hs_w[:, -H:]  >= hs_th).sum(axis=1)  >= 6
).astype(int) + (
    (pwr_w[:, -H:] >= pwr_th).sum(axis=1) >= 6
).astype(int) + (
    (ws_w[:, -H:]  >= ws_th).sum(axis=1)  >= 6
).astype(int) + (
    (mslp_w[:, -H:] <= mslp_th).sum(axis=1) >= 6
).astype(int) + (
    (dwd_cos_w[:, -H:] >= align_th).sum(axis=1) >= 4
).astype(int)

# Final labels (trend count can be used later if needed)
y = base.copy()

# --------------------------------------------------
# Sanity checks
# --------------------------------------------------
print("Columns:", df.columns.tolist())
print("Class dist:")
for name, m in [("train", train_mask), ("val", val_mask), ("test", test_mask)]:
    print(name, pd.Series(y[m]).value_counts(normalize=True).sort_index().to_dict())


Columns: ['time', 'ws', 'tp', 'hs', 'pwr', 'mslp', 'temp', 'surge', 'twl', 'steep', 'dwd_sin', 'dwd_cos', 'hs_ma3', 'hs_ma6', 'hs_ma12', 'ws_ma3', 'ws_ma6', 'ws_ma12', 'pwr_ma3', 'pwr_ma6', 'pwr_ma12', 'mslp_ma3', 'mslp_ma6', 'mslp_ma12', 'temp_ma3', 'temp_ma6', 'temp_ma12', 'surge_ma3', 'surge_ma6', 'surge_ma12', 'twl_ma3', 'twl_ma6', 'twl_ma12', 'steep_ma3', 'steep_ma6', 'steep_ma12', 'tp_ma3', 'tp_ma6', 'tp_ma12']
Class dist:
train {0: 0.725734690152414, 1: 0.1796986705606766, 2: 0.07084131909585957, 3: 0.017933740987496578, 4: 0.005791579203553284}
val {0: 0.7295774005111354, 1: 0.17819003285870755, 2: 0.06863818912011684, 3: 0.015950164293537787, 4: 0.007644213216502373}
test {0: 0.735375345217173, 1: 0.18250291009517722, 2: 0.060712573893593226, 3: 0.016889964165886836, 4: 0.0045192066281697215}


In [3]:
!pip install sktime

MA YOK

In [ ]:
import numpy as np
import numpy.lib.stride_tricks as st

from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier

# --------------------------------------------------
# Window length + raw feature list (no moving averages)
# --------------------------------------------------
W = int(W)
COLS = ["ws", "tp", "mslp", "surge", "twl", "steep", "temp", "dwd_sin", "dwd_cos"]
print("Using RAW features (no MAs):", COLS)

# --------------------------------------------------
# Build sliding windows: (num_windows, W, num_features)
# --------------------------------------------------
arrs = [df[c].to_numpy(np.float32) for c in COLS]
X_list = [st.sliding_window_view(a, window_shape=W) for a in arrs]
X = np.stack(X_list, axis=-1).astype(np.float32)

# Labels + split masks (assumed already prepared outside)
y = np.asarray(y)
train_mask = np.asarray(train_mask)
val_mask   = np.asarray(val_mask)
test_mask  = np.asarray(test_mask)

# --------------------------------------------------
# Align lengths (safe-guard if something is longer/shorter)
# --------------------------------------------------
N = min(len(X), len(y), len(train_mask), len(val_mask), len(test_mask))
if (N != len(X)) or (N != len(y)) or (N != len(train_mask)) or (N != len(val_mask)) or (N != len(test_mask)):
    print(f"Aligning lengths -> X:{len(X)} y:{len(y)} "
          f"train:{len(train_mask)} val:{len(val_mask)} test:{len(test_mask)} => N={N}")

X = X[:N]
y = y[:N]
train_mask = train_mask[:N]
val_mask   = val_mask[:N]
test_mask  = test_mask[:N]

# --------------------------------------------------
# Drop windows containing NaN/inf (keeps training clean)
# --------------------------------------------------
finite_mask = np.isfinite(X).all(axis=(1, 2))
if not finite_mask.all():
    print(f"Dropping {(~finite_mask).sum()} windows due to NaN/inf.")
    X = X[finite_mask]
    y = y[finite_mask]
    train_mask = train_mask[finite_mask]
    val_mask   = val_mask[finite_mask]
    test_mask  = test_mask[finite_mask]

# Train / val / test split
X_train, y_train = X[train_mask], y[train_mask]
X_val,   y_val   = X[val_mask],   y[val_mask]
X_test,  y_test  = X[test_mask],  y[test_mask]

n_classes = int(np.max(y_train)) + 1
print("Shapes:",
      "\n  X_train:", X_train.shape,
      "\n  X_val:  ", X_val.shape,
      "\n  X_test: ", X_test.shape,
      "\n  #classes:", n_classes)

# --------------------------------------------------
# Standardization using TRAIN only (no leakage)
# (computed over all timesteps inside train windows)
# --------------------------------------------------
eps = 1e-6
flat = X_train.reshape(-1, X_train.shape[-1])
mu  = flat.mean(axis=0, keepdims=True)
std = np.maximum(flat.std(axis=0, keepdims=True), eps)

def standardize(Xn):
    return (Xn - mu) / std

X_train = standardize(X_train).astype(np.float32)
X_val   = standardize(X_val).astype(np.float32)
X_test  = standardize(X_test).astype(np.float32)
print("Standardization: ON (train-only).")

# --------------------------------------------------
# MiniROCKET expects (n_instances, n_channels, n_timepoints)
# so we transpose: (N, W, F) -> (N, F, W)
# --------------------------------------------------
X_train_mr = np.transpose(X_train, (0, 2, 1)).astype(np.float32)
X_val_mr   = np.transpose(X_val,   (0, 2, 1)).astype(np.float32)
X_test_mr  = np.transpose(X_test,  (0, 2, 1)).astype(np.float32)

# --------------------------------------------------
# Import MiniROCKET (multivariate if available)
# --------------------------------------------------
try:
    from sktime.transformations.panel.rocket import MiniRocketMultivariate as MiniRocket
except Exception:
    from sktime.transformations.panel.rocket import MiniRocket

mr = MiniRocket(random_state=42)

print("Fitting MiniROCKET on TRAIN ...")
mr.fit(X_train_mr)

print("Transforming ...")
Z_train = mr.transform(X_train_mr)
Z_val   = mr.transform(X_val_mr)
Z_test  = mr.transform(X_test_mr)

# Convert potential pandas output to numpy
def _to_numpy(Z):
    return Z.to_numpy() if hasattr(Z, "to_numpy") else np.asarray(Z)

Z_train = _to_numpy(Z_train).astype(np.float32, copy=False)
Z_val   = _to_numpy(Z_val).astype(np.float32, copy=False)
Z_test  = _to_numpy(Z_test).astype(np.float32, copy=False)

print("Transformed shapes:", Z_train.shape, Z_val.shape, Z_test.shape)

# --------------------------------------------------
# Extra scaling in ROCKET feature space
# with_mean=False is common because ROCKET features can be sparse-ish
# --------------------------------------------------
z_scaler = StandardScaler(with_mean=False)
Z_train_s = z_scaler.fit_transform(Z_train)
Z_val_s   = z_scaler.transform(Z_val)
Z_test_s  = z_scaler.transform(Z_test)

# --------------------------------------------------
# Tune alpha (regularization strength) on validation set
# --------------------------------------------------
alphas = np.logspace(-6, -2, 9)

best_alpha = None
best_f1 = -1.0
best_clf = None

print("Selecting alpha on VAL (macro-F1) ...")
for a in alphas:
    clf = SGDClassifier(
        loss="log_loss",          # logistic regression via SGD
        alpha=float(a),           # regularization strength
        penalty="l2",
        max_iter=2000,
        tol=1e-3,
        early_stopping=True,      # internal split from train for stopping
        n_iter_no_change=5,
        validation_fraction=0.1,
        class_weight="balanced",
        random_state=42
    )
    clf.fit(Z_train_s, y_train.astype(int))
    yp = clf.predict(Z_val_s)
    f1m = f1_score(y_val.astype(int), yp, average="macro")
    print(f"  alpha={a:.2e} | val_f1_macro={f1m:.4f}")

    # Keep the best model based on validation macro-F1
    if f1m > best_f1 + 1e-6:
        best_f1 = f1m
        best_alpha = float(a)
        best_clf = clf

print(f"Best alpha: {best_alpha:.2e} | best val_f1_macro={best_f1:.4f}")

# --------------------------------------------------
# Final evaluation on each split
# --------------------------------------------------
def report_split(name, Zs, y_true):
    y_pred = best_clf.predict(Zs)
    print(f"\n{name}")
    print("F1-macro:", f1_score(y_true, y_pred, average="macro"))
    print(classification_report(y_true, y_pred, zero_division=0))
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))

report_split("TRAIN", Z_train_s, y_train.astype(int))
report_split("VAL",   Z_val_s,   y_val.astype(int))
report_split("TEST",  Z_test_s,  y_test.astype(int))


Using RAW features (no MAs): ['ws', 'tp', 'mslp', 'surge', 'twl', 'steep', 'temp', 'dwd_sin', 'dwd_cos']
Shapes: 
  X_train: (262968, 36, 9) 
  X_val:   (43824, 36, 9) 
  X_test:  (43813, 36, 9) 
  #classes: 5
Standardization: ON (train-only).
Fitting MiniROCKET on TRAIN ...
Transforming ...
Transformed shapes: (262968, 9996) (43824, 9996) (43813, 9996)
Selecting alpha on VAL (macro-F1) ...
  alpha=1.00e-06 | val_f1_macro=0.5723
  alpha=3.16e-06 | val_f1_macro=0.5439
  alpha=1.00e-05 | val_f1_macro=0.4986
  alpha=3.16e-05 | val_f1_macro=0.4799
  alpha=1.00e-04 | val_f1_macro=0.5306
  alpha=3.16e-04 | val_f1_macro=0.5381
  alpha=1.00e-03 | val_f1_macro=0.5444
  alpha=3.16e-03 | val_f1_macro=0.5303
  alpha=1.00e-02 | val_f1_macro=0.5567
Best alpha: 1.00e-06 | best val_f1_macro=0.5723

TRAIN
F1-macro: 0.5653308437628668
              precision    recall  f1-score   support

           0       0.87      0.99      0.93    190845
           1       0.70      0.43      0.53     47255
        

MA'LARI EKLEYİNCE MINIROCKET

In [ ]:
import numpy as np
import numpy.lib.stride_tricks as st

from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier

# --------------------------------------------------
# Window length
# --------------------------------------------------
W = int(W)

# --------------------------------------------------
# Raw (instantaneous) features
# --------------------------------------------------
RAW_COLS = ["ws", "tp", "mslp", "surge", "twl", "steep", "temp", "dwd_sin", "dwd_cos"]

# --------------------------------------------------
# Requested moving average features
# (only a subset of variables)
# --------------------------------------------------
MA_BASE_COLS = ["ws", "mslp", "temp", "surge", "twl"]
MA_HOURS = [3, 6, 12]

MA_COLS = [f"{c}_ma{h}" for c in MA_BASE_COLS for h in MA_HOURS]

# Keep only MA columns that actually exist in the dataframe
existing_ma_cols = [c for c in MA_COLS if c in df.columns]

# Final feature list
COLS = RAW_COLS + existing_ma_cols

print("Using RAW cols:", RAW_COLS)
print("Requested MA cols:", MA_COLS)
print("Found MA cols in df:", existing_ma_cols)
print("Final feature cols:", COLS)

# --------------------------------------------------
# Build sliding windows: (num_windows, W, num_features)
# --------------------------------------------------
arrs = [df[c].to_numpy(np.float32) for c in COLS]
X_list = [st.sliding_window_view(a, window_shape=W) for a in arrs]
X = np.stack(X_list, axis=-1).astype(np.float32)

# Labels and split masks (assumed prepared beforehand)
y = np.asarray(y)
train_mask = np.asarray(train_mask)
val_mask   = np.asarray(val_mask)
test_mask  = np.asarray(test_mask)

# --------------------------------------------------
# Align lengths (defensive check)
# --------------------------------------------------
N = min(len(X), len(y), len(train_mask), len(val_mask), len(test_mask))
if (N != len(X)) or (N != len(y)) or (N != len(train_mask)) or (N != len(val_mask)) or (N != len(test_mask)):
    print(f"Aligning lengths -> X:{len(X)} y:{len(y)} "
          f"train:{len(train_mask)} val:{len(val_mask)} test:{len(test_mask)} => N={N}")

X = X[:N]
y = y[:N]
train_mask = train_mask[:N]
val_mask   = val_mask[:N]
test_mask  = test_mask[:N]

# --------------------------------------------------
# Drop windows containing NaN or inf values
# --------------------------------------------------
finite_mask = np.isfinite(X).all(axis=(1, 2))
if not finite_mask.all():
    print(f"Dropping {(~finite_mask).sum()} windows due to NaN/inf.")
    X = X[finite_mask]
    y = y[finite_mask]
    train_mask = train_mask[finite_mask]
    val_mask   = val_mask[finite_mask]
    test_mask  = test_mask[finite_mask]

# Train / validation / test split
X_train, y_train = X[train_mask], y[train_mask]
X_val,   y_val   = X[val_mask],   y[val_mask]
X_test,  y_test  = X[test_mask],  y[test_mask]

n_classes = int(np.max(y_train)) + 1
print("Shapes:",
      "\n  X_train:", X_train.shape,
      "\n  X_val:  ", X_val.shape,
      "\n  X_test: ", X_test.shape,
      "\n  #classes:", n_classes)

# --------------------------------------------------
# Standardization using TRAIN only (no leakage)
# (statistics computed over all timesteps)
# --------------------------------------------------
eps = 1e-6
flat = X_train.reshape(-1, X_train.shape[-1])
mu  = flat.mean(axis=0, keepdims=True)
std = np.maximum(flat.std(axis=0, keepdims=True), eps)

def standardize(Xn):
    return (Xn - mu) / std

X_train = standardize(X_train).astype(np.float32)
X_val   = standardize(X_val).astype(np.float32)
X_test  = standardize(X_test).astype(np.float32)
print("Standardization: ON (train-only).")

# --------------------------------------------------
# MiniROCKET expects (instances, channels, time)
# Convert from (N, W, F) -> (N, F, W)
# --------------------------------------------------
X_train_mr = np.transpose(X_train, (0, 2, 1)).astype(np.float32)
X_val_mr   = np.transpose(X_val,   (0, 2, 1)).astype(np.float32)
X_test_mr  = np.transpose(X_test,  (0, 2, 1)).astype(np.float32)

# --------------------------------------------------
# Import MiniROCKET (multivariate if available)
# --------------------------------------------------
try:
    from sktime.transformations.panel.rocket import MiniRocketMultivariate as MiniRocket
except Exception:
    from sktime.transformations.panel.rocket import MiniRocket

mr = MiniRocket(random_state=42)

print("Fitting MiniROCKET on TRAIN ...")
mr.fit(X_train_mr)

print("Transforming ...")
Z_train = mr.transform(X_train_mr)
Z_val   = mr.transform(X_val_mr)
Z_test  = mr.transform(X_test_mr)

# Convert potential pandas output to numpy
def _to_numpy(Z):
    return Z.to_numpy() if hasattr(Z, "to_numpy") else np.asarray(Z)

Z_train = _to_numpy(Z_train).astype(np.float32, copy=False)
Z_val   = _to_numpy(Z_val).astype(np.float32, copy=False)
Z_test  = _to_numpy(Z_test).astype(np.float32, copy=False)

print("Transformed shapes:", Z_train.shape, Z_val.shape, Z_test.shape)

# --------------------------------------------------
# Extra scaling in ROCKET feature space
# (with_mean=False is standard for ROCKET features)
# --------------------------------------------------
z_scaler = StandardScaler(with_mean=False)
Z_train_s = z_scaler.fit_transform(Z_train)
Z_val_s   = z_scaler.transform(Z_val)
Z_test_s  = z_scaler.transform(Z_test)

# --------------------------------------------------
# Tune regularization strength (alpha) on validation set
# --------------------------------------------------
alphas = np.logspace(-6, -2, 9)

best_alpha = None
best_f1 = -1.0
best_clf = None

print("Selecting alpha on VAL (macro-F1) ...")
for a in alphas:
    clf = SGDClassifier(
        loss="log_loss",          # logistic regression via SGD
        alpha=float(a),           # L2 regularization strength
        penalty="l2",
        max_iter=2000,
        tol=1e-3,
        early_stopping=True,
        n_iter_no_change=5,
        validation_fraction=0.1,
        class_weight="balanced",
        random_state=42
    )
    clf.fit(Z_train_s, y_train.astype(int))
    yp = clf.predict(Z_val_s)
    f1m = f1_score(y_val.astype(int), yp, average="macro")
    print(f"  alpha={a:.2e} | val_f1_macro={f1m:.4f}")

    # Keep the best model based on validation macro-F1
    if f1m > best_f1 + 1e-6:
        best_f1 = f1m
        best_alpha = float(a)
        best_clf = clf

print(f"Best alpha: {best_alpha:.2e} | best val_f1_macro={best_f1:.4f}")

# --------------------------------------------------
# Final evaluation on each split
# --------------------------------------------------
def report_split(name, Zs, y_true):
    y_pred = best_clf.predict(Zs)
    print(f"\n{name}")
    print("F1-macro:", f1_score(y_true, y_pred, average="macro"))
    print(classification_report(y_true, y_pred, zero_division=0))
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))

report_split("TRAIN", Z_train_s, y_train.astype(int))
report_split("VAL",   Z_val_s,   y_val.astype(int))
report_split("TEST",  Z_test_s,  y_test.astype(int))


Using RAW cols: ['ws', 'tp', 'mslp', 'surge', 'twl', 'steep', 'temp', 'dwd_sin', 'dwd_cos']
Requested MA cols: ['ws_ma3', 'ws_ma6', 'ws_ma12', 'mslp_ma3', 'mslp_ma6', 'mslp_ma12', 'temp_ma3', 'temp_ma6', 'temp_ma12', 'surge_ma3', 'surge_ma6', 'surge_ma12', 'twl_ma3', 'twl_ma6', 'twl_ma12']
Found MA cols in df: ['ws_ma3', 'ws_ma6', 'ws_ma12', 'mslp_ma3', 'mslp_ma6', 'mslp_ma12', 'temp_ma3', 'temp_ma6', 'temp_ma12', 'surge_ma3', 'surge_ma6', 'surge_ma12', 'twl_ma3', 'twl_ma6', 'twl_ma12']
Final feature cols: ['ws', 'tp', 'mslp', 'surge', 'twl', 'steep', 'temp', 'dwd_sin', 'dwd_cos', 'ws_ma3', 'ws_ma6', 'ws_ma12', 'mslp_ma3', 'mslp_ma6', 'mslp_ma12', 'temp_ma3', 'temp_ma6', 'temp_ma12', 'surge_ma3', 'surge_ma6', 'surge_ma12', 'twl_ma3', 'twl_ma6', 'twl_ma12']
Dropping 11 windows due to NaN/inf.
Shapes: 
  X_train: (262957, 36, 24) 
  X_val:   (43824, 36, 24) 
  X_test:  (43813, 36, 24) 
  #classes: 5
Standardization: ON (train-only).
Fitting MiniROCKET on TRAIN ...
Transforming ...
Transf